# Day 9 - Smoke Basin

>These caves seem to be lava tubes. Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly settles like rain.
>
>If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).
>
>Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:
>
>```
2199943210
3987894921
9856789892
8767896789
9899965678
>```
>
>Each number corresponds to the height of a particular location, where 9 is the highest and 0 is the lowest a location can be.
>
>Your first goal is to find the low points - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)
>
>In the above example, there are four low points, all highlighted: two are in the first row (a 1 and a 0), one is in the third row (a 5), and one is in the bottom row (also a 5). All other locations on the heightmap have some lower adjacent location, and so are not low points.
>
>The risk level of a low point is 1 plus its height. In the above example, the risk levels of the low points are 2, 1, 6, and 6. The sum of the risk levels of all low points in the heightmap is therefore 15.

Finally a easy-to-understand, short description! I'm excited.

I recently mentored a junior with a similar kind of problem (for a completely different problem domain, bitmap rendering) so this seems fun.

In [4]:
from utils import read_input

def transformer(input_line):
    return [int(num) for num in list(input_line)]

heightmap = read_input(9, transformer)

## Part 1

> Find all of the low points on your heightmap. **What is the sum of the risk levels of all low points on your heightmap?**

In [42]:
def find_neighbors(heightmap, indices):
    i, j = indices
    
    top = i == 0
    bottom = i == len(heightmap) - 1
    middle = not (top or bottom)
    
    left = j == 0
    right = j == len(heightmap[i]) - 1
    center = not (left or right)
    
    if top:
        if left:
            return [
                heightmap[i][j+1],
                heightmap[i+1][j],
                heightmap[i+1][j+1]
            ]
        elif right:
            return [
                heightmap[i][j-1],
                heightmap[i+1][j],
                heightmap[i+1][j-1],            
            ]
        else:
            return [
                heightmap[i][j-1],
                heightmap[i][j+1],
                heightmap[i+1][j-1],
                heightmap[i+1][j],
                heightmap[i+1][j+1],
            ]
    elif middle:
        if left:
            return [
                heightmap[i-1][j],
                heightmap[i-1][j+1],
                heightmap[i][j+1],
                heightmap[i+1][j],
                heightmap[i+1][j+1],
            ]
        elif right:
            return [
                heightmap[i-1][j-1],
                heightmap[i-1][j],
                heightmap[i][j-1],
                heightmap[i+1][j-1],
                heightmap[i+1][j],
            ]
        else:
            return [
                heightmap[i-1][j-1],
                heightmap[i-1][j],
                heightmap[i-1][j+1],
                heightmap[i][j-1],
                heightmap[i][j+1],
                heightmap[i+1][j-1],
                heightmap[i+1][j],
                heightmap[i+1][j+1],
            ]
    elif bottom:
        if left:
            return [
                heightmap[i-1][j],
                heightmap[i-1][j+1],
                heightmap[i][j+1],
            ]
        elif right:
            return [
                heightmap[i-1][j-1],
                heightmap[i-1][j],
                heightmap[i][j-1],
            ]
        else:
            return [
                heightmap[i-1][j-1],
                heightmap[i-1][j],
                heightmap[i-1][j+1],
                heightmap[i][j-1],
                heightmap[i][j+1],
            ]
    
low_points = []
for i in range(len(heightmap)):
    for j in range(len(heightmap[i])):
        neighbors = find_neighbors(heightmap, (i, j))
        if all(n > heightmap[i][j] for n in neighbors):
            low_points.append(heightmap[i][j])

result = sum(x+1 for x in low_points)
print(f'Solution: {result}')
assert result == 545 # For refactoring
    

Solution: 545


## Part 2

>Next, you need to find the largest basins so you know what areas are most important to avoid.
>
>A basin is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height 9 do not count as being in any basin, and all other locations will always be part of exactly one basin.
>
>The size of a basin is the number of locations within the basin, including the low point. The example above has four basins.

_Editor's note: these basins don't show up in my markdown._

>
>The top-left basin, size 3:
>```
2199943210
3987894921
9856789892
8767896789
9899965678
>```
>
>The top-right basin, size 9:
>```
2199943210
3987894921
9856789892
8767896789
9899965678
>```
>
>The middle basin, size 14:
>```
2199943210
3987894921
9856789892
8767896789
9899965678
>```
>
>The bottom-right basin, size 9:
>```
2199943210
3987894921
9856789892
8767896789
9899965678
>```
>
>Find the three largest basins and multiply their sizes together. In the above example, this is 9 * 14 * 9 = 1134.
>
>What do you get if you multiply together the sizes of the three largest basins?